In [2]:
import cv2
import full_makeup
import align_faces
import dlib
import os
import eyecolor_convert
from flask import Flask, flash, Response, request, redirect, url_for, render_template

from werkzeug.utils import secure_filename


ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

app = Flask(__name__)
detector = dlib.get_frontal_face_detector()

UPLOAD_FOLDER = 'static/uploads/'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024



#for cctv camera use rtsp://username:password@ip_address:554/user=username_password='password'_channel=channel_number_stream=0.sdp' instead of camera
def gen_frames0():  # generate frame by frame from camera
    while True:
        # Capture frame-by-frame
        success, frame = camera.read()  # read the camera frame
        frame = cv2.resize(frame,(int(256),int(256)))

        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result
def gen_frames1(your_image):  # generate frame by frame from camera
    # Capture frame-by-frame
    frame = cv2.imread(your_image)  # read the camera frame
    frame = cv2.resize(frame,(int(256),int(256)))
    ret, buffer = cv2.imencode('.jpg', frame)
    frame = buffer.tobytes()
    yield (b'--frame\r\n'
           b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result


def gen_frames2(your_image = None, skin_image = None, lip_image=None,eye_image=None):  # generate frame by frame from camera
    a=0
    #frame = eyecolor_convert.eyecolor_make(your_image,[255,0,0])
    frame = cv2.imread(your_image)
    frame = cv2.resize(frame,(int(256),int(256)))
    #frame = eyecolor_convert.eyecolor_make(frame,[255,0,0])
    temp = frame[:,:,::-1].copy()
    dets = detector(temp, 1)
    if len(dets) != 0:
        temp_face = align_faces.align_faces(temp)
        src_img = temp_face[0]
        faces1 = detector(src_img,1)
        if len(faces1) !=0:
            frame, final_output_no, makeup_frame =full_makeup.full_makeup(frame ,skin_image, lip_image, eye_image)
            cv2.imwrite('test2.jpg',makeup_frame)
            a=1
            return makeup_frame
        else:
            if a == 0:
                return frame
            else:
                return makeup_frame
    else:
        if a == 0:
                return frame
        else:
             return makeup_frame
                    
def gen_frames3(your_image):  # generate frame by frame from camera
    a=0
    # Capture frame-by-frame
    frame = cv2.imread(your_image)
    frame = cv2.resize(frame,(int(256),int(256)))
    temp = frame[:,:,::-1].copy()
    dets = detector(temp, 1)
    if len(dets) != 0:
        temp_face = align_faces.align_faces(temp)
        src_img = temp_face[0]
        faces1 = detector(src_img,1)
        if len(faces1) !=0:
            src_img = src_img[:,:,::-1].copy()
            ret, buffer = cv2.imencode('.jpg', src_img)
            makeup_frame = buffer.tobytes()
            a=1
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + makeup_frame + b'\r\n')  # concat frame one by one and show result
        else:
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            if a == 0:
                yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result
            else:
                yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + makeup_frame + b'\r\n')  # concat frame one by one and show result
    else:
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        if a == 0:
            yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result
        else:
            yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + makeup_frame + b'\r\n')  # concat frame one by one and show result



def final(you,skin,lip,eye):
    global eyes 
    eyes = eyecolor_convert.eyecolor_make(you,[255,0,0])
    #frame =gen_frames2(you,skin,lip,eye)
    ret, buffer = cv2.imencode('.jpg',eyes)
    frame = buffer.tobytes()
    yield (b'--frame\r\n'
        b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def upload_form():
    return render_template('final.html')


@app.route('/', methods=['POST'])
def upload_image():
    if 'file' not in request.files or 'file1' not in request.files or 'file2' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    file1 = request.files['file1']
    file2 = request.files['file2']
    file3 = request.files['file3']
    if file.filename == '' or file1.filename =='' or file2.filename ==''or file3.filename =='':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename) and file1 and allowed_file(file1.filename) and file2 and allowed_file(file2.filename) and file3 and allowed_file(file3.filename):
        global filename
        filename = secure_filename(file.filename)
        global filename1
        filename1 = secure_filename(file1.filename)
        global filename2
        filename2 = secure_filename(file2.filename)
        global filename3
        filename3 = secure_filename(file3.filename)
        global RR
        global GG
        global BB
        RR = request.form['R']
        GG= request.form['G']
        BB= request.form['B']
        
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        file1.save(os.path.join(app.config['UPLOAD_FOLDER'], filename1))
        file2.save(os.path.join(app.config['UPLOAD_FOLDER'], filename2))
        file3.save(os.path.join(app.config['UPLOAD_FOLDER'], filename3))
        #print('upload_image filename: ' + filename)
        flash('Image successfully uploaded and displayed')
        return render_template('final.html', filename=filename, filename1=filename1, filename2=filename2, filename3=filename3)

@app.route('/display/<filename3>')
def video_feed0(filename3):
    """Video streaming route. Put this in the src attribute of an img tag."""
    return redirect(url_for('static', filename='uploads/' + filename3), code=301)

@app.route('/display/<filename>')
def display_image(filename):
    #print('display_image filename: ' + filename0)
    return redirect(url_for('static', filename='uploads/' + filename), code=301)

@app.route('/display/<filename1>')
def display_image1(filename1):
    #print('display_image filename: ' + filename0)
    return redirect(url_for('static', filename='uploads/' + filename1), code=301)

@app.route('/display/<filename2>')
def display_image2(filename2):
    #print('display_image filename: ' + filename0)
    return redirect(url_for('static', filename='uploads/' + filename2), code=301)


@app.route('/video_feed3') #align
def video_feed3():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen_frames3('static/uploads/'+filename3),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed4')
def video_feed4():
    if str(filename) != "NONE.png":
        skin_image = 'static/uploads/'+str(filename)
    else:
        skin_image =None
        
    if str(filename1) != "NONE.png":
        lip_image = 'static/uploads/'+str(filename1)
    else:
        lip_image =None
        
    if str(filename2) != "NONE.png":
        eye_image = 'static/uploads/'+str(filename2)
    else:
        eye_image =None
    
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(final('static/uploads/'+filename3,skin_image,lip_image,eye_image),
                    mimetype='multipart/x-mixed-replace; boundary=frame')


if __name__ == "__main__":
    app.run()#host="192.168.123.109",port=8888)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Sep/2020 12:40:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2020 12:40:31] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2020 12:40:31] "GET /display/ HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2020 12:40:31] "GET /video_feed3?filename3= HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2020 12:41:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2020 12:41:05] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2020 12:41:08] "GET /video_feed3?filename3=36ED8CED-87F0-443C-9743-1D0972362DCE.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2020 12:41:09] "GET /video_feed4 HTTP/1.1" 500 -
Error on request:
Traceback (most recent call last):
  File "C:\Users\jkpar\.conda\envs\pytorch\Lib\site-packages\werkzeug\serving.py", line 323, in run_wsgi
    execute(self.server.app)
  File "C:\Users\jkpar\.conda\envs\pytorch\Lib\site-packages\werkzeug\serving.py", line 314, in execute
    for data in application_iter:
  File "C:\U

In [ ]:
filename3